In [8]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [5]:
dataset_path = '../../data/benchmark/lt2_windows__cvs_gt2.csv'

n_preds_path = '../../data/benchmark/preds/n_bilstm.npy'
c_preds_path = '../../data/benchmark/preds/c_bilstm.npy'

In [3]:
data_df = pd.read_csv(dataset_path)
data_df.head()

,protein,cleavages
0,AAADSAQWLSVKEETIFLHDGLIRVTDLAELPSEILGAPEAADTDL...,"[(52, 63), (17, 25), (52, 62)]"
1,ACAWLEAQEEEEVGFPVRPQVPLRPMTYKAAVDLSHFLKEKGGLEG...,"[(29, 37), (141, 150), (141, 149)]"
2,AFAQRALSDSLSRRLRRHVPTHQSRRRSLGHLSPTARRACEDAIRC...,"[(108, 116), (92, 100), (103, 112), (74, 82), ..."
3,AWRRRRSGTSGKATWWCSGLRRASPTPSRRVQSWATAVMWKPSPSS...,"[(224, 232), (225, 233), (219, 232), (65, 73)]"
4,DHVASCGVNLYQFYGPSGQFTHEFDGDEQFYVDLEKKETAWRWPEF...,"[(38, 46), (22, 31), (22, 32), (41, 49), (22, ..."


In [7]:
n_preds = np.load(n_preds_path, allow_pickle=True)
c_preds = np.load(c_preds_path, allow_pickle=True)

n_preds.shape, c_preds.shape

((867,), (867,))

In [11]:
cleavages = data_df['cleavages'].apply(literal_eval).values
seq_lens = data_df['protein'].apply(len).values

In [ ]:
n_targets = [np.zeros(seq_len + 1) for seq_len in seq_lens]
c_targets = [np.zeros(seq_len + 1) for seq_len in seq_lens]

for i, cleavage in enumerate(cleavages):
    for n, c in cleavage:
        n_targets[i][n - 1] = 1
        c_targets[i][c - 1] = 1